In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
import plotly.express as px
from sklearn.manifold import TSNE

In [ ]:
# -------------------------------
# 1️⃣ Carregar MNIST
# -------------------------------
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist.data, mnist.target.astype(int)  # X shape: (70000, 784)
print("Shape do dataset:", X.shape)

In [ ]:
# -------------------------------
# 2️⃣ Visualizar alguns dígitos aleatórios
# -------------------------------
plt.figure(figsize=(10, 4))
indices = np.random.choice(len(X), 10, replace=False)
for i, idx in enumerate(indices):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X[idx].reshape(28, 28), cmap='gray')
    plt.title(y[idx])
    plt.axis('off')
plt.suptitle("Dígitos aleatórios do MNIST")
plt.show()

In [ ]:
# -------------------------------
# 3️⃣ PCA e gráfico da variância explicada
# -------------------------------
pca = PCA()
X_pca = pca.fit_transform(X)

explained_variance = pca.explained_variance_ratio_

plt.figure(figsize=(8, 4))
plt.plot(np.cumsum(explained_variance)*100, marker='.')
plt.xlabel("Número de componentes")
plt.ylabel("Variância acumulada (%)")
plt.title("Variância explicada acumulada")
plt.grid(True)
plt.show()

In [ ]:
# -------------------------------
# 4️⃣ Scatterplot 2D (PC1 x PC2)
# -------------------------------
df_2d = pd.DataFrame(X_pca[:, :2], columns=['PC1', 'PC2'])
df_2d['digit'] = y

plt.figure(figsize=(8, 6))
sns.scatterplot(data=df_2d.sample(2000), x='PC1', y='PC2', hue='digit', palette='tab10', legend='full', alpha=0.7)
plt.title("Scatterplot 2D dos dois primeiros componentes principais (amostra 2000)")
plt.show()

In [ ]:
# -------------------------------
# 5️⃣ Scatterplot 3D (PC1 x PC2 x PC3)
# -------------------------------
# Para não sobrecarregar o navegador, vamos usar uma amostra
sample_indices = np.random.choice(len(X_pca), 2000, replace=False)
df_3d = pd.DataFrame({
    'PC1': X_pca[sample_indices, 0],
    'PC2': X_pca[sample_indices, 1],
    'PC3': X_pca[sample_indices, 2],
    'digit': y[sample_indices].astype(str)
})

# Scatter 3D interativo com Plotly
fig = px.scatter_3d(
    df_3d,
    x='PC1', y='PC2', z='PC3',
    color='digit',
    color_discrete_sequence=px.colors.qualitative.Set1,
    title="Scatterplot 3D dos três primeiros componentes principais (MNIST)",
    width=800, height=600
)

fig.update_traces(marker=dict(size=2, opacity=0.7))
fig.show()

In [ ]:
# -------------------------------
# 6️⃣ Reconstrução das imagens com n componentes
# -------------------------------
n_components = 50  # Pode mudar para qualquer número ≤ 784
pca_reduced = PCA(n_components=n_components)
X_reduced = pca_reduced.fit_transform(X)
X_reconstructed = pca_reduced.inverse_transform(X_reduced)

# Visualizar alguns dígitos reconstruídos
plt.figure(figsize=(12, 4))
for i, idx in enumerate(np.random.choice(len(X), 10, replace=False)):
    # Original
    plt.subplot(2, 10, i + 1)
    plt.imshow(X[idx].reshape(28, 28), cmap='gray')
    plt.axis('off')
    # Reconstruído
    plt.subplot(2, 10, i + 11)
    plt.imshow(X_reconstructed[idx].reshape(28, 28), cmap='gray')
    plt.axis('off')
plt.suptitle(f"Comparação: Original (linha superior) vs Reconstruído com {n_components} PCs (linha inferior)")
plt.show()

In [ ]:
# -------------------------------
# 7️⃣ Visualizar as componentes principais
# -------------------------------

# Vamos usar o PCA já ajustado com todos os componentes
# X_pca já foi feito anteriormente com PCA() fit_transform(X)
pca_full = PCA()
pca_full.fit(X)

# Extrair os 25 primeiros componentes principais
components = pca_full.components_[:25]  # shape (25, 784)

# Plotar como imagens 28x28
plt.figure(figsize=(10, 10))
for i, comp in enumerate(components):
    plt.subplot(5, 5, i + 1)
    plt.imshow(comp.reshape(28, 28), cmap='gray')
    plt.title(f"PC{i+1}")
    plt.axis('off')

plt.suptitle("25 Primeiras Componentes Principais do MNIST", fontsize=16)
plt.tight_layout()
plt.show()


## Outra redução de dimensão t-SNE

In [ ]:
# -------------------------------
# 1️⃣ Amostragem para t-SNE
# -------------------------------
sample_size = 5000  # t-SNE é lento, vamos usar 5000 amostras
indices = np.random.choice(len(X), sample_size, replace=False)
X_sample = X[indices]
y_sample = y[indices]

In [ ]:
# -------------------------------
# 2️⃣ t-SNE 2D
# -------------------------------
tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=1000)
X_tsne = tsne.fit_transform(X_sample)


In [ ]:
# -------------------------------
# 3️⃣ Criar DataFrame para plot
# -------------------------------
df_tsne = pd.DataFrame({
    'TSNE1': X_tsne[:, 0],
    'TSNE2': X_tsne[:, 1],
    'digit': y_sample.astype(str)
})

In [ ]:
# -------------------------------
# 4️⃣ Plot 2D
# -------------------------------
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df_tsne, x='TSNE1', y='TSNE2', hue='digit', palette='tab10', alpha=0.7, legend='full')
plt.title("MNIST 2D usando t-SNE")
plt.show()